# Part 16 - Lesson 2 - Notebook

In [1]:
import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np

In [2]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/irish-lyrics-eof.txt \
    -O /tmp/irish-lyrics-eof.txt

--2021-06-24 15:22:24--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/irish-lyrics-eof.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.203.128, 74.125.204.128, 64.233.188.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.203.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68970 (67K) [text/plain]
Saving to: ‘/tmp/irish-lyrics-eof.txt’

/tmp/irish-lyrics-e 100%[===================>]  67.35K  --.-KB/s    in 0.004s  

2021-06-24 15:22:25 (17.7 MB/s) - ‘/tmp/irish-lyrics-eof.txt’ saved [68970/68970]



In [3]:
tokenizer = Tokenizer()

data = open('/tmp/irish-lyrics-eof.txt').read()

corpus = data.lower().split("\n")

tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

print(tokenizer.word_index)
print(total_words)

{'pardon': 2636, 'hats': 2124, 'vermin': 1484, 'follol': 1995, 'yellow': 2104, 'four': 536, 'hath': 1098, 'woods': 1073, 'sleep': 458, 'hanging': 1119, 'ringlets': 729, 'captain': 692, 'yonder': 1069, 'marching': 2071, 'shure': 790, 'whose': 1485, 'swan': 2145, 'sweetest': 968, 'edward': 2096, 'under': 737, 'lord': 517, 'sorry': 1915, 'pride': 768, 'whack': 547, 'sinking': 447, 'lure': 1903, 'tarry': 2456, 'chanters': 1882, 'rise': 1058, 'shoot': 2672, 'every': 345, 'moons': 2563, 'elevations': 1773, 'thems': 1668, 'doubts': 2304, 'strayed': 1074, 'wooden': 2127, 'flute': 1431, 'venus': 2287, 'clothes': 615, 'eileen': 429, 'parted': 1244, 'nigh': 2045, 'tired': 726, 'bubbling': 2032, 'bacon': 1815, 'preacher': 1618, 'rusty': 1301, 'street': 293, 'sustaining': 1252, 'faithful': 2466, 'sailed': 849, 'n': 1789, 'shining': 280, 'blue': 264, 'roddy': 357, 'solemn': 1235, 'ned': 1050, 'fingers': 1010, 'mccorley': 358, 'assembled': 2208, 'ringin': 1451, 'above': 333, 'new': 188, 'murther': 18

In [4]:
input_sequences = []
for line in corpus:
	token_list = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)

# pad sequences
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]

ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [5]:
print(tokenizer.word_index['in'])
print(tokenizer.word_index['the'])
print(tokenizer.word_index['town'])
print(tokenizer.word_index['of'])
print(tokenizer.word_index['athy'])
print(tokenizer.word_index['one'])
print(tokenizer.word_index['jeremy'])
print(tokenizer.word_index['lanigan'])

8
1
71
6
713
39
1790
1791


In [6]:
print(xs[6])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 2]


In [7]:
print(ys[6])

[0. 0. 0. ... 0. 0. 0.]


In [8]:
print(xs[5])
print(ys[5])

[   0    0    0    0    0    0    0    0    0   51   12   96 1217   48
    2]
[0. 0. 0. ... 0. 0. 0.]


In [9]:
print(tokenizer.word_index)

{'pardon': 2636, 'hats': 2124, 'vermin': 1484, 'follol': 1995, 'yellow': 2104, 'four': 536, 'hath': 1098, 'woods': 1073, 'sleep': 458, 'hanging': 1119, 'ringlets': 729, 'captain': 692, 'yonder': 1069, 'marching': 2071, 'shure': 790, 'whose': 1485, 'swan': 2145, 'sweetest': 968, 'edward': 2096, 'under': 737, 'lord': 517, 'sorry': 1915, 'pride': 768, 'whack': 547, 'sinking': 447, 'lure': 1903, 'tarry': 2456, 'chanters': 1882, 'rise': 1058, 'shoot': 2672, 'every': 345, 'moons': 2563, 'elevations': 1773, 'thems': 1668, 'doubts': 2304, 'strayed': 1074, 'wooden': 2127, 'flute': 1431, 'venus': 2287, 'clothes': 615, 'eileen': 429, 'parted': 1244, 'nigh': 2045, 'tired': 726, 'bubbling': 2032, 'bacon': 1815, 'preacher': 1618, 'rusty': 1301, 'street': 293, 'sustaining': 1252, 'faithful': 2466, 'sailed': 849, 'n': 1789, 'shining': 280, 'blue': 264, 'roddy': 357, 'solemn': 1235, 'ned': 1050, 'fingers': 1010, 'mccorley': 358, 'assembled': 2208, 'ringin': 1451, 'above': 333, 'new': 188, 'murther': 18

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(total_words, activation='softmax'))
adam = Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
# earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
history = model.fit(xs, ys, epochs=100, verbose=1)
# print model.summary()
print(model)

Train on 12038 samples
Epoch 1/100
12038/12038 [==============================] - 15s 1ms/sample - loss: 6.6393 - accuracy: 0.0736
Epoch 2/100
12038/12038 [==============================] - 7s 623us/sample - loss: 5.7175 - accuracy: 0.1159
Epoch 3/100
12038/12038 [==============================] - 8s 629us/sample - loss: 4.8357 - accuracy: 0.1648
Epoch 4/100
12038/12038 [==============================] - 8s 624us/sample - loss: 3.8917 - accuracy: 0.2461
Epoch 5/100
12038/12038 [==============================] - 8s 627us/sample - loss: 3.0715 - accuracy: 0.3439
Epoch 6/100
12038/12038 [==============================] - 7s 621us/sample - loss: 2.6332 - accuracy: 0.4237
Epoch 7/100
12038/12038 [==============================] - 7s 621us/sample - loss: 2.7244 - accuracy: 0.4213
Epoch 8/100
12038/12038 [==============================] - 7s 622us/sample - loss: 2.1194 - accuracy: 0.5130
Epoch 9/100
12038/12038 [==============================] - 8s 623us/sample - loss: 1.7129 - accuracy: 0.59

In [ ]:
import matplotlib.pyplot as plt

def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.show()

In [ ]:
plot_graphs(history, 'acc')

KeyError: ignored

In [ ]:
seed_text = "I've got a bad feeling about this"
next_words = 100

for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted = model.predict_classes(token_list, verbose=0)
	output_word = ""
	for word, index in tokenizer.word_index.items():
		if index == predicted:
			output_word = word
			break
	seed_text += " " + output_word
print(seed_text)